## Argo File Loader

In [ ]:
# try with newest snapshot
# try with ftp that gmaze used in issue, probably going to be really slow

In [1]:
import xarray as xr
import numpy as np
import gsw
import matplotlib.pyplot as plt
import matplotlib as mpl
from importlib import reload
from cmocean import cm as cmo
import xrft
import pandas as pd
import argopy
from argopy import DataFetcher
import dask

In [2]:
print('argopy: {}, xarray: {}, dask: {}'.format(argopy.__version__, xr.__version__, dask.__version__))

argopy: 1.0.0, xarray: 2024.2.0, dask: 2024.10.0


In [3]:
import os
os.chdir('/home.ufs/amf2288/argo-intern/funcs')
import density_funcs as df
import EV_funcs as ef
import filt_funcs as ff
import plot_funcs as pf
import processing_funcs as prf

In [4]:
reload(df)
reload(ef)
reload(ff)
reload(pf)
#reload(mf)
reload(prf)

<module 'processing_funcs' from '/home/amf2288/argo-intern/funcs/processing_funcs.py'>

## testing parallel loading with dask client

In [ ]:
from dask.distributed import Client

In [ ]:
client= Client(processes=True)

In [ ]:
print(client)

In [ ]:
box = [-27.5, -22.5, -5, 5, 0, 2001]

In [ ]:
%%time
with argopy.set_options(parallel=client):
    f = DataFetcher(src='gdac', ftp='/swot/SUM05/dbalwada/Argo_sync', progress=True).region(box)
    print('%i chunks to process' % len(f.uri))
    print(f)
    ds = f.load().data#.to_xarray()

Okay the process is running on multiple GPUs, in this case 10-12 machines. However, obviously there are a ton of error messages---I'm not really sure what's going wrong. The box I've started with is really small, so I don't think there should be a memory error. Not sure where to go from here.

## Load by box (coordinate: N_PROF)

In [ ]:
#box=[lon_min,lon_max,lat_min,lat_max,depth_min,depth_max]
box_atl = [-25,-20,-70,70,0,2001]
box_pac = [-180,-175,-70,70,0,2001]
box_watl = [-60,-55,10,45,0,2001]
box_wpac = [150,155,-5,50,0,2001]
box_wind = [60,65,-65,25,0,2001]

In [ ]:
NW = [-180,0,0,90,0,2001]
NE = [0,180,-90,0,0,2001]
SW = [-180,0,-90,0,0,2001]
SE = [0,-180,-90,0,0,2001]

In [5]:
box1 = [-180,-90,  0,90,0,2001]
box2 = [- 90,  0,  0,90,0,2001]
box3 = [   0, 90,  0,90,0,2001]
box4 = [  90,180,  0,90,0,2001]
box5 = [-180,-90,-90, 0,0,2001]
box6 = [- 90,  0,-90, 0,0,2001]
box7 = [   0, 90,-90, 0,0,2001]
box8 = [  90,180,-90, 0,0,2001]

In [ ]:
#pf.plot_box([NW,NE,SW,SE])

In [6]:
ds = prf.get_box(box3,2)

loading points complete


/home.ufs/amf2288/mambaforge-pypy3/envs/Argo_Nov_24/lib/python3.12/site-packages/argopy/data_fetchers/gdac_data.py:331: UserWarning: Found more than 50 files to load, this may take a while to process sequentially ! Consider using another data source (eg: 'erddap') or the 'parallel=True' option to improve processing time.
  warnings.warn(
 25%|██▌       | 481/1898 [27:08<1:36:47,  4.10s/it]

Error: 502, message='Proxy Error', url='https://data-argo.ifremer.fr/dac/aoml/5905088/5905088_prof.nc'


100%|██████████| 1898/1898 [1:43:30<00:00,  3.27s/it]


to xarray complete
point to profile complete
interpolation complete
adding spice complete


IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
ds

In [ ]:
ds = ds.assign_attrs({"Fetched_uri":''})
ds

In [ ]:
ds.to_netcdf("/swot/SUM05/amf2288/sync-boxes/lon:({},{})_lat:({},{})_ds_z.nc".format(NW[0],NW[1],NW[2],NW[3]))

In [ ]:
box_1x = [-10,0,-10,0,0,2000]   #error
box_1y = [-10,0,-20,-10,0,2000] #done
box_1z = [-10,0,-30,-20,0,2000] #done
box_1a = [-10,0,-40,-30,0,2000] #done
box_1b = [-10,0,-50,-40,0,2000] #done
box_1c = [-10,0,-60,-50,0,2000] #done
box_1d = [-10,0,-70,-60,0,2000] #done
box_long=[-10,-8,-70,-10,0,2000]

In [ ]:
pf.plot_box([box_1x,box_1y,box_1z,box_1a,box_1b,box_1c,box_1d,box_long])

In [ ]:
ds_z=ff.get_box(box_long,4)

In [ ]:
ds['N_PROF'].attrs = {'name':'Fetched Profile Number'}
ds['PRES_INTERPOLATED'].attrs = {'name':'Interpolated Pressure', 'units':'dbar', 'notes':'interpolated to standard pressure grid of 2m'}

ds['N_PROF_NEW'].attrs = {'name':'Sequential Profile Number'}
ds['LATITUDE'].attrs = {'name':'Latitude', 'units':'°N'}
ds['LONGITUDE'].attrs = {'name':'Longitude', 'units':'°E'}
ds['month'].attrs = {'name':'Month'}
ds['year'].attrs = {'name':'Year'}
ds['MLD'].attrs = {'name':'Mixed Layer Depth', 'units':'m'}

ds['CT'].attrs = {'name':'Conservative Temperature', 'units':'°C'}
ds['SA'].attrs = {'name':'Absolute Salinity', 'units':'g kg-1'}
ds['SIG0'].attrs = {'name':'Potential Density', 'units':'kg m-1', 'notes':'referenced to 0 dbar'}
ds['SPICE'].attrs = {'name':'Spiciness', 'units':'kg m-1', 'notes':'referenced to 0 dbar'}                  
ds = ds.assign_attrs({"Fetched_uri":''})

In [ ]:
ds_z

In [ ]:
plt.scatter(ds_z.LONGITUDE, ds_z.LATITUDE)

In [ ]:
ds_z.to_netcdf("202206_boxes/lon:({},{})_lat:({},{})_ds_z.nc".format(box_long[0],box_long[1],box_long[2],box_long[3]))

In [ ]:
erddap_loader=ArgoDataFetcher(src'erddap',parallel=True,progress=True)

In [ ]:
gmaze_loader=ArgoDataFetcher(src='gdac',ftp='https://data-argo.ifremer.fr/',parallel=True,progress=True)

In [ ]:
usa_loader=ArgoDataFetcher(src='gdac',ftp='ftp://usgodae.org/pub/outgoing/argo',parallel=True,progress=True)

In [ ]:
ds=argo_loader.region(box_1x)
print('loading points complete')
ds=ds.to_xarray()
print('to xarray complete')

In [ ]:
ds=ds.argo.teos10(['CT','SA','SIG0'])
ds=ds.argo.point2profile()
print('point to profile complete')

In [ ]:
def test_get_box(box,sample_min):
    '''Takes latitude/longitude/depth data and a sample rate and returns an xarray with CT, SA, SIG0, and SPICE interpolated to a pressure grid of 2m. 
    
    box: lat/lon in the form: box=[lon_min, lon_max, lat_min, lat_max, depth_min, depth_max]
    sample_min: minimum sample rate [m]
    '''
    
    ds=argo_loader.region(box)
    print('loading points complete')
    
    ds=ds.to_xarray()
    print('to xarray complete')
    
    ds=ds.argo.teos10(['CT','SA','SIG0'])
    ds=ds.argo.point2profile()
    print('point to profile complete')
    
    ds_interp=get_ds_interp(ds,0,2000,sample_min)
    print('interpolation complete')
    
    ds_interp['SPICE'] = gsw.spiciness0(ds_interp.SA,ds_interp.CT).rename('SPICE')
    print('adding spice complete')
        
    return ds_interp

In [ ]:
test = test_get_box(box_medi,4)

In [ ]:
ds=argo_loader.region(box_medi)

In [ ]:
ds = ds.data

In [ ]:
ds

In [ ]:
ds.PSAL.dropna('N_POINTS')

In [ ]:
for n in range(0,len(ds.N_PROF)):
    

In [ ]:
'''
methods are also functions, but specific to the class they're applied to
class (animals) --> subclass (dogs) --> object (golden retriever)
open_mfdataset() from xarray, for opening data in multiple netcdfs
1) write loop opening all files yourself, check for PSAL, if not there remove file
2) go into open_mfdataset() and see if there's a loop there to exploit instead

locally install repo (pip install -e??)
uses the code from the local repo instead
then make changes to repo: print statement in part of the code we think is happening
then work on loops, etc.


'''

In [ ]:
def to_xarray(self, errors: str = "ignore"):
        """ Load Argo data and return a :class:`xarray.Dataset`

        Parameters
        ----------
        errors: str, default='ignore'
            Define how to handle errors raised during data URIs fetching:

                - 'ignore' (default): Do not stop processing, simply issue a debug message in logging console
                - 'silent':  Do not stop processing and do not issue log message
                - 'raise': Raise any error encountered

        Returns
        -------
        :class:`xarray.Dataset`
        """
        if (
            len(self.uri) > 50
            and isinstance(self.method, str)
            and self.method == "sequential"
        ):
            warnings.warn(
                "Found more than 50 files to load, this may take a while to process sequentially ! "
                "Consider using another data source (eg: 'erddap') or the 'parallel=True' option to improve processing time."
            )
        elif len(self.uri) == 0:
            raise DataNotFound("No data found for: %s" % self.indexfs.cname)

        # Download data:
        ds = self.fs.open_mfdataset(
            self.uri, #list of all float files?
            method=self.method,
            concat_dim="N_POINTS",
            concat=True,
            preprocess=self._preprocess_multiprof,
            progress=self.progress,
            errors=errors,
            open_dataset_opts={'xr_opts': {'decode_cf': 1, 'use_cftime': 0, 'mask_and_scale': 1}},
        )

        # Data post-processing:
        ds["N_POINTS"] = np.arange(
            0, len(ds["N_POINTS"])
        )  # Re-index to avoid duplicate values
        ds = ds.set_coords("N_POINTS")
        ds = ds.sortby("TIME")

        # Remove netcdf file attributes and replace them with simplified argopy ones:
        ds.attrs = {}
        if self.dataset_id == "phy":
            ds.attrs["DATA_ID"] = "ARGO"
        if self.dataset_id == "bgc":
            ds.attrs["DATA_ID"] = "ARGO-BGC"
        ds.attrs["DOI"] = "http://doi.org/10.17882/42182"
        ds.attrs["Fetched_from"] = self.server
        ds.attrs["Fetched_by"] = getpass.getuser()
        ds.attrs["Fetched_date"] = pd.to_datetime("now", utc=True).strftime("%Y/%m/%d")
        ds.attrs["Fetched_constraints"] = self.cname()
        if len(self.uri) == 1:
            ds.attrs["Fetched_uri"] = self.uri[0]
        else:
            ds.attrs["Fetched_uri"] = ";".join(self.uri)

        return ds

In [ ]:
print(ds_z.SIG0.values.min(), ds_z.SIG0.values.max())

In [ ]:
rho_grid = np.linspace(26.6, 28, 1001)

In [ ]:
ds_rho = df.interpolate2density_prof(ds_z, rho_grid)

In [ ]:
ds_rho

In [ ]:
ds_rho.to_netcdf("202206_boxes/lon:({},{})_lat:({},{})_ds_rho.nc".format(box[0],box[1],box[2],box[3]))

## Load by float (coordinate: distance)

In [ ]:
float_ID = 6901265

In [ ]:
ds_z = ff.get_float(float_ID, 4)

In [ ]:
ds_z = ds_z.assign_attrs({"Fetched_uri":''})

In [ ]:
ds_z

In [ ]:
ds_z.to_netcdf("202206_floats/float_ID:({})_ds_z.nc".format(float_ID))

In [ ]:
print(ds_z.SIG0.values.min(), ds_z.SIG0.values.max())

In [ ]:
rho_grid = np.linspace(26.4, 28, 1001)

In [ ]:
ds_rho = df.interpolate2density_prof(ds_z, rho_grid)

In [ ]:
ds_rho

In [ ]:
ds_rho.to_netcdf("202206_floats/float_ID:({})_ds_rho.nc".format(float_ID))

## Loading Boxes by Degree

In [ ]:
lons = np.arange(-180,181,4)
lats = np.arange(-90,91,4)

boxes=[]

for n in range(0,len(lons)-1):
    for m in range(0,len(lats-1)):
        lon_min = lons[n]
        lon_max = lons[n+1]
        lat_min = lats[n]
        lat_max = lats[n+1]
        box_n = np.array([lon_min, lon_max, lat_min, lat_max])
        boxes.append(box_n)

In [ ]:
n=0
lons[n+1]

In [ ]:
from argopy import IndexFetcher as ArgoIndexFetcher

box_bad=[-15,-14,36.5,37]#, '2000-01-01', '2021-06'] # need to remove pressure poinds because the region function for this index fetcher seems a bit different.

idx = ArgoIndexFetcher(src='gdac',dataset='phy',mode='standard',
                            ftp="/swot/SUM05/dbalwada/202203-ArgoData").region(box_bad).load()
idx.index

In [ ]:
float_IDs = [1900041,1900749,]